# **NLP2_Run1**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [ ]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [ ]:
problem = """A buyer needs to acquire 239,600,480 units of a product and is considering bids from five suppliers, labeled A through E, each of whom can only supply a portion of the total required amount.
Each vendor has proposed different pricing structures, incorporating both setup fees and variable unit costs that change based on the quantity ordered.

The buyer's objective is to allocate the order among these suppliers to minimize overall costs, accounting for both setup and unit costs.

Vendor A offers a set up cost of $3855.34 and a unit cost of $61.150 per thousand of units.
Vendor A can supply up to 33 million units.

Vendor B offers a set up cost of $125,804.84 if purchasing between 22,000,000-70,000,000 units from vendor B with a unit cost of $68.099 per thousand units.
If purchasing between 70,000,001-100,000,000 units from vendor B, the set up cost increases to $269304.84 and the unit cost sinks to $66.049 per thousand units.
If purchasing between 100,000,001-150,000,000 units from vendor B, the unit cost per thousand units further decreases to $64.099, but the set up cost increases to $464304.84.
If purchasing between 150,000,001 and 160,000,000 units from vendor B, the unit cost is $62.119 per thousand units and the set up cost equals $761304.84.

Vendor C offers set up costs of $13,456.00 and a unit cost of $62.019 per thousand units.
Vendor C can supply up to 165.6 million units. Vendor D offers set up costs of $6,583.98 and a unit cost of $72.488 for a set of thousand units.

Vendor D can supply up to 12 million units at a price of $72.488 per thousand units and with a set up cost of $6583.98.

Vendor E offers free set up if purchasing between 0 and 42 million units of vendor E with a unit price of $70.150 per thousand units.
If purchasing between 42,000,001 and 77 million units from vendor E, the unit cost starts at $68.150 per thousand units, but with every one million units purchased the price decreases at a rate of 0.05 percent. An additional set up cost of $84000 will be charged as well.

Note that zero units may be purchased from vendor B: otherwise no positive number of units less than 22,000,000 may be purchased."""

In [ ]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [ ]:
print(response1.choices[0].message.content)

1. Xa = Units purchased from Vendor A
2. Xb = Units purchased from Vendor B
3. Xc = Units purchased from Vendor C
4. Xd = Units purchased from Vendor D
5. Xe = Units purchased from Vendor E
6. Sa = Setup cost for Vendor A
7. Sb = Setup cost for Vendor B
8. Sc = Setup cost for Vendor C
9. Sd = Setup cost for Vendor D
10. Se = Setup cost for Vendor E
11. Ca = Unit cost for Vendor A
12. Cb = Unit cost for Vendor B
13. Cc = Unit cost for Vendor C
14. Cd = Unit cost for Vendor D
15. Ce = Unit cost for Vendor E
16. TB = Total units purchasing from Vendor B, for deciding setup cost 
17. TE = Total units purchasing from Vendor E, for deciding setup cost and unit cost decrement rate
Note: Sa, Sb, Sc, Sd, Se, Ca, Cb, Cc, Cd, Ce are decision variables those will be determined based on total units purchasing from Vendors B and E.


In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
print(response2.choices[0].message.content)

Minimize Z = Sa*Xa + Ca*Xa/1000 + Sb*Xb + Cb*Xb/1000 + Sc*Xc + Cc*Xc/1000 + Sd*Xd + Cd*Xd/1000 + Se*Xe + Ce*Xe/1000

Where:
Z = Total cost
Xa, Xb, Xc, Xd, Xe = units purchased from vendors A, B, C, D, E, respectively
Sa, Sb, Sc, Sd, Se = setup costs for vendors A, B, C, D, E, respectively
Ca, Cb, Cc, Cd, Ce = costs per thousand units from vendors A, B, C, D, E, respectively
Note that the setup costs and unit costs are decision variables and are determined based on the total units purchased from vendors B and E (TB and TE).


In [ ]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
print(response3.choices[0].message.content)

1. Xa >= 0
2. Xb >= 0
3. Xc >= 0
4. Xd >= 0
5. Xe >= 0

6. Xa <= 33,000,000
7. Xc <= 165,600,000
8. Xd <= 12,000,000
9. Xe <= 77,000,000

Quantity order from each vendor:
10. 22,000,000 <= Xb <= 160,000,000

Total demand:
11. Xa + Xb + Xc + Xd + Xe = 239,600,480

Set up and unit cost conditions for vendor B:
12. 125804.84 <= Sb <= 761304.84
13. 68.099 <= Cb <= 62.119

Set up and unit cost conditions for vendor E:
14. Se >= 0 if Xe <= 42,000,000
15. Se = 84000 if Xe > 42,000,000
16. Ce = 70.150 if Xe <= 42,000,000
17. 68.15 <= Ce <= 70.15 if Xe > 42,000,000

Fixed set up and unit cost for vendors A, C and D:
18. Sa = 3855.34
19. Sc = 13456
20. Sd = 6583.98
21. Ca = 61.15
22. Cc = 62.019
23. Cd = 72.488


### **Generate Pyomo Code**

In [ ]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
Markdown(response4.choices[0].message.content)

Here is a Python program using the Pyomo library to solve this optimization problem:

```python
# Import the Pyomo package
from pyomo.environ import *

# Create a model object
model = ConcreteModel()

# Define decision variables
model.xa = Var(within=NonNegativeReals)
model.xb = Var(within=NonNegativeReals)
model.xc = Var(within=NonNegativeReals)
model.xd = Var(within=NonNegativeReals)
model.xe = Var(within=NonNegativeReals)
model.sb = Var(within=NonNegativeReals)
model.se = Var(within=NonNegativeReals)
model.cb = Var(within=NonNegativeReals)
model.ce = Var(within=NonNegativeReals)

# Set the limits of decision variables
model.xa.setub(33000000)
model.xb.setlb(22000000); model.xb.setub(160000000)
model.xc.setub(165600000)
model.xd.setub(12000000)
model.xe.setub(77000000)
model.sb.setlb(125804.84); model.sb.setub(761304.84)
model.cb.setlb(68.099); model.cb.setub(62.119)
model.se.setub(84000)
model.ce.setlb(68.15); model.ce.setub(70.15)


# Define the objective function
model.cost = Objective(expr = model.xa*(3855.34+61.15/1000) + model.xb*(model.sb + model.cb/1000) + model.xc*(13456+62.019/1000) +
    model.xd*(6583.98+72.488/1000) + model.xe*(model.se+model.ce/1000), sense=minimize)

# Add constraints
model.demand = Constraint(expr = model.xa + model.xb + model.xc + model.xd + model.xe == 239600480)

model.cost_conditions_b = ConstraintList()
model.cost_conditions_b.add(inequality(125804.84, model.sb, 761304.84))
model.cost_conditions_b.add(inequality(62.119, model.cb, 68.099))

model.cost_conditions_e = ConstraintList()
model.cost_conditions_e.add(model.se == 0 if model.xe <= 42000000 else model.se == 84000)
model.cost_conditions_e.add(model.ce == 70.150 if model.xe <= 42000000 else inequality(68.15, model.ce, 70.15)) 
```

Please remember that I am assuming that the costs are linear functions with their rates and that they do not have more complicated forms.

### **Run the code Generated by GPT4**

In [ ]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 37.2 MB/s eta 0:00:00


In [27]:
_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["coin"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [ ]:
from pyomo.environ import *

# Create a model object
model = ConcreteModel()

# Define decision variables
model.xa = Var(within=NonNegativeReals)
model.xb = Var(within=NonNegativeReals)
model.xc = Var(within=NonNegativeReals)
model.xd = Var(within=NonNegativeReals)
model.xe = Var(within=NonNegativeReals)
model.sb = Var(within=NonNegativeReals)
model.se = Var(within=NonNegativeReals)
model.cb = Var(within=NonNegativeReals)
model.ce = Var(within=NonNegativeReals)

# Set the limits of decision variables
model.xa.setub(33000000)
model.xb.setlb(22000000); model.xb.setub(160000000)
model.xc.setub(165600000)
model.xd.setub(12000000)
model.xe.setub(77000000)
model.sb.setlb(125804.84); model.sb.setub(761304.84)
model.cb.setlb(68.099); model.cb.setub(62.119)
model.se.setub(84000)
model.ce.setlb(68.15); model.ce.setub(70.15)


# Define the objective function
model.cost = Objective(expr = model.xa*(3855.34+61.15/1000) + model.xb*(model.sb + model.cb/1000) + model.xc*(13456+62.019/1000) +
    model.xd*(6583.98+72.488/1000) + model.xe*(model.se+model.ce/1000), sense=minimize)

# Add constraints
model.demand = Constraint(expr = model.xa + model.xb + model.xc + model.xd + model.xe == 239600480)

model.cost_conditions_b = ConstraintList()
model.cost_conditions_b.add(inequality(125804.84, model.sb, 761304.84))
model.cost_conditions_b.add(inequality(62.119, model.cb, 68.099))

model.cost_conditions_e = ConstraintList()
model.cost_conditions_e.add(model.se == 0 if model.xe <= 42000000 else model.se == 84000)
model.cost_conditions_e.add(model.ce == 70.150 if model.xe <= 42000000 else inequality(68.15, model.ce, 70.15))

PyomoException: Cannot convert non-constant Pyomo expression (xe  <=  42000000) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [28]:
from pyomo.environ import *

# Create a model object
model = ConcreteModel()

# Define decision variables
model.xa = Var(within=NonNegativeReals)
model.xb = Var(within=NonNegativeReals)
model.xc = Var(within=NonNegativeReals)
model.xd = Var(within=NonNegativeReals)
model.xe = Var(within=NonNegativeReals)
model.sb = Var(within=NonNegativeReals)
model.se = Var(within=NonNegativeReals)
model.cb = Var(within=NonNegativeReals)
model.ce = Var(within=NonNegativeReals)

# Set the limits of decision variables
model.xa.setub(33000000)
model.xb.setlb(22000000); model.xb.setub(160000000)
model.xc.setub(165600000)
model.xd.setub(12000000)
model.xe.setub(77000000)
model.sb.setlb(125804.84); model.sb.setub(761304.84)
model.cb.setlb(68.099); model.cb.setub(62.119)
model.se.setub(84000)
model.ce.setlb(68.15); model.ce.setub(70.15)


# Define the objective function
model.cost = Objective(expr = model.xa*(3855.34+61.15/1000) + model.xb*(model.sb + model.cb/1000) + model.xc*(13456+62.019/1000) +
    model.xd*(6583.98+72.488/1000) + model.xe*(model.se+model.ce/1000), sense=minimize)

# Add constraints
model.demand = Constraint(expr = model.xa + model.xb + model.xc + model.xd + model.xe == 239600480)

model.cost_conditions_b = ConstraintList()
model.cost_conditions_b.add(inequality(125804.84, model.sb, 761304.84))
model.cost_conditions_b.add(inequality(62.119, model.cb, 68.099))

model.cost_conditions_e = ConstraintList()
model.y = Var(domain=Binary)

M = 100000000  # A large number, greater than any expected value of xe


model.C1 = Constraint(expr=model.xe <= 42000000 + M * model.y)
model.C2 = Constraint(expr=model.xe >= 42000000 - M * (1 - model.y))

model.C3 = Constraint(expr=model.se == 0 * (1 - model.y))
model.C4 = Constraint(expr=model.se == 84000 * model.y)

model.xe_switch = Constraint(expr=model.xe <= 42000000 + M * (1 - model.y))
model.xe_switch2 = Constraint(expr=model.xe >= 42000000 - M * model.y)

# Constraint for ce based on the value of xe
model.ce_condition = Constraint(expr=model.ce == 70.15 * model.y + (68.15 * (1 - model.y)))
model.ce_range = Constraint(expr=inequality(68.15, model.ce, 70.15))

SolverFactory('couenne').solve(model)

  - termination condition: infeasible
  - message from solver: Couenne (/tmp/tmpvlvq4nwx.pyomo.nl Feb 21 2023)\x3a Infeasible


{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 0, 'Sense': 'unknown'}], 'Solver': [{'Status': 'warning', 'Message': 'Couenne (/tmp/tmpvlvq4nwx.pyomo.nl Feb 21 2023)\\x3a Infeasible', 'Termination condition': 'infeasible', 'Id': 220, 'Error rc': 0, 'Time': 0.011655569076538086}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}